In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('aggreations').getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/10/03 18:27:25 WARN Utils: Your hostname, deepak-ubuntu, resolves to a loopback address: 127.0.1.1; using 192.168.31.105 instead (on interface wlp1s0)
25/10/03 18:27:25 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/10/03 18:27:30 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/10/03 18:27:35 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
25/10/03 18:27:35 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
25/10/03 18:27:35 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.
25/10/03 18:27:35 WARN Uti

In [24]:
spark

In [25]:
df = spark.read.option('header', 'true').option('inferSchema', 'true').format('csv').load('C:\\Users\\DEEPAK REDDY\\OneDrive\\Documents\\BigData\\Spark-The-Definitive-Guide-master\\data\\retail-data\\all\\*.csv')

In [26]:
df.cache()

DataFrame[InvoiceNo: string, StockCode: string, Description: string, Quantity: int, InvoiceDate: string, UnitPrice: double, CustomerID: int, Country: string]

In [27]:
df.explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- InMemoryTableScan [InvoiceNo#2412, StockCode#2413, Description#2414, Quantity#2415, InvoiceDate#2416, UnitPrice#2417, CustomerID#2418, Country#2419]
      +- InMemoryRelation [InvoiceNo#2412, StockCode#2413, Description#2414, Quantity#2415, InvoiceDate#2416, UnitPrice#2417, CustomerID#2418, Country#2419], StorageLevel(disk, memory, deserialized, 1 replicas)
            +- FileScan csv [InvoiceNo#17,StockCode#18,Description#19,Quantity#20,InvoiceDate#21,UnitPrice#22,CustomerID#23,Country#24] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex(1 paths)[file:/C:/Users/DEEPAK REDDY/OneDrive/Documents/BigData/Spark-The-Defin..., PartitionFilters: [], PushedFilters: [], ReadSchema: struct<InvoiceNo:string,StockCode:string,Description:string,Quantity:int,InvoiceDate:string,UnitP...




In [28]:
df.createOrReplaceTempView('dfTable')

In [82]:
%load_ext sparksql_magic

The sparksql_magic extension is already loaded. To reload it, use:
  %reload_ext sparksql_magic


In [30]:
%%sparksql
select * from dfTable

only showing top 5 row(s)


InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850,United Kingdom
536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850,United Kingdom
536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850,United Kingdom
536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850,United Kingdom
536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850,United Kingdom


In [31]:
%%sparksql
select country, count(*) as count from dfTable group by country order by count desc 

only showing top 5 row(s)


country,count
United Kingdom,495478
Germany,9495
France,8557
EIRE,8196
Spain,2533


In [32]:
%%sparksql
select country, round(sum(Quantity * UnitPrice), 2) as total_spend from dfTable group by country order by total_spend desc

only showing top 5 row(s)


country,total_spend
United Kingdom,8187806.36
Netherlands,284661.54
EIRE,263276.82
Germany,221698.21
France,197403.9


In [33]:
df.show(truncate=False)

+---------+---------+-----------------------------------+--------+--------------+---------+----------+--------------+
|InvoiceNo|StockCode|Description                        |Quantity|InvoiceDate   |UnitPrice|CustomerID|Country       |
+---------+---------+-----------------------------------+--------+--------------+---------+----------+--------------+
|536365   |85123A   |WHITE HANGING HEART T-LIGHT HOLDER |6       |12/1/2010 8:26|2.55     |17850     |United Kingdom|
|536365   |71053    |WHITE METAL LANTERN                |6       |12/1/2010 8:26|3.39     |17850     |United Kingdom|
|536365   |84406B   |CREAM CUPID HEARTS COAT HANGER     |8       |12/1/2010 8:26|2.75     |17850     |United Kingdom|
|536365   |84029G   |KNITTED UNION FLAG HOT WATER BOTTLE|6       |12/1/2010 8:26|3.39     |17850     |United Kingdom|
|536365   |84029E   |RED WOOLLY HOTTIE WHITE HEART.     |6       |12/1/2010 8:26|3.39     |17850     |United Kingdom|
|536365   |22752    |SET 7 BABUSHKA NESTING BOXES       

In [34]:
df.count()

541909

In [35]:
from pyspark.sql import functions as fn
from pyspark.sql import types as ty

In [36]:
df.select(fn.count('*')).show()

+--------+
|count(1)|
+--------+
|  541909|
+--------+



In [37]:
df.select(fn.count_distinct(fn.col('StockCode'))).show()

+-------------------------+
|count(DISTINCT StockCode)|
+-------------------------+
|                     4070|
+-------------------------+



In [39]:
df.select(fn.approx_count_distinct(fn.col('StockCode'), 0.1)).show()

+--------------------------------+
|approx_count_distinct(StockCode)|
+--------------------------------+
|                            3364|
+--------------------------------+



In [40]:
df.select(fn.first(fn.col('StockCode')), fn.last(fn.col('StockCode'))).show()

+----------------+---------------+
|first(StockCode)|last(StockCode)|
+----------------+---------------+
|          85123A|          22138|
+----------------+---------------+



In [41]:
df.select(fn.min('Quantity'), fn.max('Quantity')).show()

+-------------+-------------+
|min(Quantity)|max(Quantity)|
+-------------+-------------+
|       -80995|        80995|
+-------------+-------------+



In [42]:
df.select(fn.sum('Quantity')).show()

+-------------+
|sum(Quantity)|
+-------------+
|      5176450|
+-------------+



In [43]:
df.select(fn.sum_distinct('Quantity')).show()

+----------------------+
|sum(DISTINCT Quantity)|
+----------------------+
|                 29310|
+----------------------+



In [44]:
testDf = spark.createDataFrame([(1, 2, 3), (1, 2, 3), (1, 2, 4)], schema=ty.StructType([
    ty.StructField('a', ty.StringType(), True),
    ty.StructField('b', ty.StringType(), True),
    ty.StructField('c', ty.StringType(), True)
]))

In [45]:
testDf.show()

+---+---+---+
|  a|  b|  c|
+---+---+---+
|  1|  2|  3|
|  1|  2|  3|
|  1|  2|  4|
+---+---+---+



In [46]:
testDf.select(fn.sum_distinct('c')).show()

+---------------+
|sum(DISTINCT c)|
+---------------+
|            7.0|
+---------------+



In [47]:
df.select(fn.avg('Quantity')).show()

+----------------+
|   avg(Quantity)|
+----------------+
|9.55224954743324|
+----------------+



In [48]:
df.select(
    fn.count('Quantity').alias('total_transcation'),
    fn.sum('Quantity').alias('total_purchases'),
    fn.avg('Quantity').alias('avg_purchases'),
    fn.expr('mean(Quantity)').alias('mean_purchases')
).selectExpr(
    'total_purchases/total_transcation',
    'avg_purchases',
    'mean_purchases'
).show()

+-------------------------------------+----------------+----------------+
|(total_purchases / total_transcation)|   avg_purchases|  mean_purchases|
+-------------------------------------+----------------+----------------+
|                     9.55224954743324|9.55224954743324|9.55224954743324|
+-------------------------------------+----------------+----------------+



In [22]:
# Variance and Standard Deviation

In [49]:
df.select(
    fn.var_pop('Quantity'), fn.var_samp('Quantity'),
    fn.stddev_pop('Quantity'), fn.stddev_samp('Quantity')
).show()

+------------------+------------------+--------------------+---------------------+
| var_pop(Quantity)|var_samp(Quantity)|stddev_pop(Quantity)|stddev_samp(Quantity)|
+------------------+------------------+--------------------+---------------------+
|47559.303646609165|47559.391409298856|   218.0809566344782|    218.0811578502344|
+------------------+------------------+--------------------+---------------------+



In [24]:
#skewness and kurtosis

In [50]:
df.select(fn.skewness('Quantity'), fn.kurtosis('Quantity')).show()

+-------------------+------------------+
| skewness(Quantity)|kurtosis(Quantity)|
+-------------------+------------------+
|-0.2640755761052369|119768.05495536828|
+-------------------+------------------+



In [26]:
# Covariance and Correlation

In [27]:
df.select(fn.corr('InvoiceNo','Quantity'), fn.covar_samp('InvoiceNo','Quantity'), fn.covar_pop('InvoiceNo', 'Quantity')).show()

+-------------------------+-------------------------------+------------------------------+
|corr(InvoiceNo, Quantity)|covar_samp(InvoiceNo, Quantity)|covar_pop(InvoiceNo, Quantity)|
+-------------------------+-------------------------------+------------------------------+
|     4.912186085630787E-4|             1052.7280543892425|            1052.7260778731384|
+-------------------------+-------------------------------+------------------------------+



In [28]:
#set and list

In [51]:
df.select(fn.collect_set('Country'), fn.collect_list('Country')).show()

+--------------------+---------------------+
|collect_set(Country)|collect_list(Country)|
+--------------------+---------------------+
|[Portugal, Italy,...| [United Kingdom, ...|
+--------------------+---------------------+



In [52]:
df.select(fn.size(fn.collect_set('Country')), fn.size(fn.collect_list('Country'))).show()

+--------------------------+---------------------------+
|size(collect_set(Country))|size(collect_list(Country))|
+--------------------------+---------------------------+
|                        38|                     541909|
+--------------------------+---------------------------+



In [53]:
df.agg(fn.collect_set('Country'), fn.collect_list('Country')).show()

+--------------------+---------------------+
|collect_set(Country)|collect_list(Country)|
+--------------------+---------------------+
|[Portugal, Italy,...| [United Kingdom, ...|
+--------------------+---------------------+



In [32]:
# Grouping

In [54]:
df.groupBy('InvoiceNo', 'CustomerId').count().show()

+---------+----------+-----+
|InvoiceNo|CustomerId|count|
+---------+----------+-----+
|   536846|     14573|   76|
|   537026|     12395|   12|
|   537883|     14437|    5|
|   538068|     17978|   12|
|   538279|     14952|    7|
|   538800|     16458|   10|
|   538942|     17346|   12|
|  C539947|     13854|    1|
|   540096|     13253|   16|
|   540530|     14755|   27|
|   541225|     14099|   19|
|   541978|     13551|    4|
|   542093|     17677|   16|
|   536596|      NULL|    6|
|   537252|      NULL|    1|
|   538041|      NULL|    1|
|   537159|     14527|   28|
|   537213|     12748|    6|
|   538191|     15061|   16|
|  C539301|     13496|    1|
+---------+----------+-----+
only showing top 20 rows



In [55]:
# Grouping with Expressions
df.groupby('InvoiceNo').agg(
    fn.count('Quantity').alias('quan'),
    fn.expr('count(Quantity) as count')
).show()

+---------+----+-----+
|InvoiceNo|quan|count|
+---------+----+-----+
|   536596|   6|    6|
|   536938|  14|   14|
|   537252|   1|    1|
|   537691|  20|   20|
|   538041|   1|    1|
|   538184|  26|   26|
|   538517|  53|   53|
|   538879|  19|   19|
|   539275|   6|    6|
|   539630|  12|   12|
|   540499|  24|   24|
|   540540|  22|   22|
|  C540850|   1|    1|
|   540976|  48|   48|
|   541432|   4|    4|
|   541518| 101|  101|
|   541783|  35|   35|
|   542026|   9|    9|
|   542375|   6|    6|
|   536597|  28|   28|
+---------+----+-----+
only showing top 20 rows



In [35]:
# Grouping with Maps

In [36]:
df.groupBy('InvoiceNo').agg(fn.expr('avg(quantity)'), fn.expr('stddev_pop(quantity)')).show()

+---------+------------------+--------------------+
|InvoiceNo|     avg(quantity)|stddev_pop(quantity)|
+---------+------------------+--------------------+
|   536596|               1.5|  1.1180339887498947|
|   536938|33.142857142857146|  20.698023172885524|
|   537252|              31.0|                 0.0|
|   537691|              8.15|   5.597097462078001|
|   538041|              30.0|                 0.0|
|   538184|12.076923076923077|   8.142590198943392|
|   538517|3.0377358490566038|  2.3946659604837897|
|   538879|21.157894736842106|  11.811070444356483|
|   539275|              26.0|  12.806248474865697|
|   539630|20.333333333333332|  10.225241100118645|
|   540499|              3.75|  2.6653642652865788|
|   540540|2.1363636363636362|  1.0572457590557278|
|  C540850|              -1.0|                 0.0|
|   540976|10.520833333333334|   6.496760677872902|
|   541432|             12.25|  10.825317547305483|
|   541518| 23.10891089108911|  20.550782784878713|
|   541783|1

In [37]:
# #window functions
# Spark supports three kinds of window functions: ranking functions, analytic functions,
# and aggregate functions.

In [56]:
df.show(truncate=False)

+---------+---------+-----------------------------------+--------+--------------+---------+----------+--------------+
|InvoiceNo|StockCode|Description                        |Quantity|InvoiceDate   |UnitPrice|CustomerID|Country       |
+---------+---------+-----------------------------------+--------+--------------+---------+----------+--------------+
|536365   |85123A   |WHITE HANGING HEART T-LIGHT HOLDER |6       |12/1/2010 8:26|2.55     |17850     |United Kingdom|
|536365   |71053    |WHITE METAL LANTERN                |6       |12/1/2010 8:26|3.39     |17850     |United Kingdom|
|536365   |84406B   |CREAM CUPID HEARTS COAT HANGER     |8       |12/1/2010 8:26|2.75     |17850     |United Kingdom|
|536365   |84029G   |KNITTED UNION FLAG HOT WATER BOTTLE|6       |12/1/2010 8:26|3.39     |17850     |United Kingdom|
|536365   |84029E   |RED WOOLLY HOTTIE WHITE HEART.     |6       |12/1/2010 8:26|3.39     |17850     |United Kingdom|
|536365   |22752    |SET 7 BABUSHKA NESTING BOXES       

In [39]:
# To demonstrate, we will add a date column that will convert our invoice date into a column that
# contains only date information

In [65]:
dfWithDate = df.withColumn('date', fn.to_date(fn.to_timestamp(fn.col('InvoiceDate'), 'M/d/y H:mm')))

In [67]:
dfWithDate.createOrReplaceTempView('dfWithDate')

In [72]:
dfWithDate.show(truncate=False)

+---------+---------+-----------------------------------+--------+--------------+---------+----------+--------------+----------+
|InvoiceNo|StockCode|Description                        |Quantity|InvoiceDate   |UnitPrice|CustomerID|Country       |date      |
+---------+---------+-----------------------------------+--------+--------------+---------+----------+--------------+----------+
|536365   |85123A   |WHITE HANGING HEART T-LIGHT HOLDER |6       |12/1/2010 8:26|2.55     |17850     |United Kingdom|2010-12-01|
|536365   |71053    |WHITE METAL LANTERN                |6       |12/1/2010 8:26|3.39     |17850     |United Kingdom|2010-12-01|
|536365   |84406B   |CREAM CUPID HEARTS COAT HANGER     |8       |12/1/2010 8:26|2.75     |17850     |United Kingdom|2010-12-01|
|536365   |84029G   |KNITTED UNION FLAG HOT WATER BOTTLE|6       |12/1/2010 8:26|3.39     |17850     |United Kingdom|2010-12-01|
|536365   |84029E   |RED WOOLLY HOTTIE WHITE HEART.     |6       |12/1/2010 8:26|3.39     |17850 

In [68]:
from pyspark.sql.window import Window

In [69]:
windowSpec = Window.partitionBy(fn.col('CustomerId'), fn.col('date'))\
.orderBy(fn.desc(fn.col('Quantity')))\
.rowsBetween(Window.unboundedPreceding, Window.currentRow)

In [45]:
# Now we want to use an aggregation function to learn more about each specific customer. An
# example might be establishing the maximum purchase quantity over all time.

In [70]:
maxPurchaseQuantity = fn.max(fn.col('Quantity')).over(windowSpec)

In [47]:
#rank and dense_rank

In [71]:
purchaseRank = fn.rank().over(windowSpec)

In [73]:
purchaseDenseRank = fn.dense_rank().over(windowSpec)

In [50]:
dfWithDate.columns

['InvoiceNo',
 'StockCode',
 'Description',
 'Quantity',
 'InvoiceDate',
 'UnitPrice',
 'CustomerID',
 'Country',
 'date']

In [74]:
runningTotalQuantity = fn.sum(fn.col('Quantity')).over(windowSpec)

In [75]:
rownumber = fn.row_number().over(windowSpec)

In [77]:
dfWithDate.where('CustomerId is not null').orderBy('CustomerID')\
.select(
    fn.col('CustomerId'),
    fn.col('date'),
    fn.col('Quantity'),
    maxPurchaseQuantity.alias('maxPurchaseQuantity'),
    purchaseDenseRank.alias('purchaseDenseRank'),
    purchaseRank.alias('purchaseRank'),
    runningTotalQuantity.alias('runningTotalQuantity'),
    rownumber.alias('rownumber') #.where('rownumber = 1')
).show(truncate=False)

+----------+----------+--------+-------------------+-----------------+------------+--------------------+---------+
|CustomerId|date      |Quantity|maxPurchaseQuantity|purchaseDenseRank|purchaseRank|runningTotalQuantity|rownumber|
+----------+----------+--------+-------------------+-----------------+------------+--------------------+---------+
|12346     |2011-01-18|74215   |74215              |1                |1           |74215               |1        |
|12347     |2010-12-07|36      |36                 |1                |1           |36                  |1        |
|12347     |2011-01-26|24      |24                 |1                |1           |24                  |1        |
|12347     |2011-04-07|240     |240                |1                |1           |240                 |1        |
|12347     |2011-06-09|36      |36                 |1                |1           |36                  |1        |
|12347     |2011-08-02|36      |36                 |1                |1         

In [90]:
%config SparkSql.limit=20

In [91]:
%%sparksql
SELECT CustomerId, date, Quantity, rank(Quantity) OVER (PARTITION BY CustomerId, date ORDER BY Quantity DESC NULLS LAST ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) as rank,
dense_rank(Quantity) OVER (PARTITION BY CustomerId, date ORDER BY Quantity DESC NULLS LAST ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) as dRank,
max(Quantity) OVER (PARTITION BY CustomerId, date ORDER BY Quantity DESC NULLS LAST ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) as maxPurchase
FROM dfWithDate WHERE CustomerId IS NOT NULL ORDER BY CustomerId

only showing top 20 row(s)


CustomerId,date,Quantity,rank,dRank,maxPurchase
12346,2011-01-18,74215,1,1,74215
12346,2011-01-18,-74215,2,2,74215
12347,2010-12-07,36,1,1,36
12347,2010-12-07,30,2,2,36
12347,2010-12-07,24,3,3,36
12347,2010-12-07,12,4,4,36
12347,2010-12-07,12,4,4,36
12347,2010-12-07,12,4,4,36
12347,2010-12-07,12,4,4,36
12347,2010-12-07,12,4,4,36


In [54]:
# -- in SQL
# SELECT CustomerId, date, Quantity,
# rank(Quantity) OVER (PARTITION BY CustomerId, date
# ORDER BY Quantity DESC NULLS LAST
# ROWS BETWEEN
# UNBOUNDED PRECEDING AND
# CURRENT ROW) as rank,
# dense_rank(Quantity) OVER (PARTITION BY CustomerId, date
# ORDER BY Quantity DESC NULLS LAST
# ROWS BETWEEN
# UNBOUNDED PRECEDING AND
# CURRENT ROW) as dRank,
# max(Quantity) OVER (PARTITION BY CustomerId, date
# ORDER BY Quantity DESC NULLS LAST
# ROWS BETWEEN
# UNBOUNDED PRECEDING AND
# CURRENT ROW) as maxPurchase
# FROM dfWithDate WHERE CustomerId IS NOT NULL ORDER BY CustomerId

In [92]:
#Test Date

my_data = spark.createDataFrame([('2023-10-01','2023-10-02', '2023-10-03'), ('01/10/2023', '02/10/2023', '03/10/2023'), ('10/01/2023', '10/02/2023', '10/03/2023')], )

In [93]:
new_df = spark.createDataFrame([('2023-10-01','01/10/2023', '2023-10-01'), ('2023-10-02', '02/10/2023', '2023-10-02'), ('2023-10-03', '03/10/2023', '2023-10-03')])

In [94]:
new_df.show()

+----------+----------+----------+
|        _1|        _2|        _3|
+----------+----------+----------+
|2023-10-01|01/10/2023|2023-10-01|
|2023-10-02|02/10/2023|2023-10-02|
|2023-10-03|03/10/2023|2023-10-03|
+----------+----------+----------+



In [95]:
new_df.columns

['_1', '_2', '_3']

In [105]:
new_df.withColumn('date', fn.to_date(fn.col('_1'), 'yyyy-MM-dd'))\
    .withColumn('date_string', fn.expr("date_format(date, 'y-MM-d')"))\
    .show()

+----------+----------+----------+----------+-----------+
|        _1|        _2|        _3|      date|date_string|
+----------+----------+----------+----------+-----------+
|2023-10-01|01/10/2023|2023-10-01|2023-10-01|  2023-10-1|
|2023-10-02|02/10/2023|2023-10-02|2023-10-02|  2023-10-2|
|2023-10-03|03/10/2023|2023-10-03|2023-10-03|  2023-10-3|
+----------+----------+----------+----------+-----------+



In [106]:
new_df.withColumn('date', fn.to_date(fn.col('_1'), 'yyyy-MM-dd')).show()

+----------+----------+----------+----------+
|        _1|        _2|        _3|      date|
+----------+----------+----------+----------+
|2023-10-01|01/10/2023|2023-10-01|2023-10-01|
|2023-10-02|02/10/2023|2023-10-02|2023-10-02|
|2023-10-03|03/10/2023|2023-10-03|2023-10-03|
+----------+----------+----------+----------+



Grouping Sets

#### Grouping sets are a low-level tool for combining sets of aggregations together. They give you the
#### ability to create arbitrary aggregation in their group-by statements.

In [108]:
dfWithDate.show(truncate=False)

+---------+---------+-----------------------------------+--------+--------------+---------+----------+--------------+----------+
|InvoiceNo|StockCode|Description                        |Quantity|InvoiceDate   |UnitPrice|CustomerID|Country       |date      |
+---------+---------+-----------------------------------+--------+--------------+---------+----------+--------------+----------+
|536365   |85123A   |WHITE HANGING HEART T-LIGHT HOLDER |6       |12/1/2010 8:26|2.55     |17850     |United Kingdom|2010-12-01|
|536365   |71053    |WHITE METAL LANTERN                |6       |12/1/2010 8:26|3.39     |17850     |United Kingdom|2010-12-01|
|536365   |84406B   |CREAM CUPID HEARTS COAT HANGER     |8       |12/1/2010 8:26|2.75     |17850     |United Kingdom|2010-12-01|
|536365   |84029G   |KNITTED UNION FLAG HOT WATER BOTTLE|6       |12/1/2010 8:26|3.39     |17850     |United Kingdom|2010-12-01|
|536365   |84029E   |RED WOOLLY HOTTIE WHITE HEART.     |6       |12/1/2010 8:26|3.39     |17850 

In [109]:
dfNoNull = dfWithDate.drop()

In [110]:
dfNoNull.createOrReplaceTempView('dfNoNull')

In [115]:
%%sparksql
select customerid, stockcode, sum(quantity) from dfNoNull
group by customerid, stockcode
order by customerid desc, stockcode desc limit 5

customerid,stockcode,sum(quantity)
18287,85173,48
18287,85040A,48
18287,85039B,120
18287,85039A,96
18287,84920,4


In [119]:
%%sparksql
select customerid, stockcode, sum(quantity) from dfNoNull
group by customerid, stockcode Grouping Sets((customerid, stockcode, invoiceno))
order by customerid desc, stockcode desc limit 5

customerid,stockcode,sum(quantity)
18287,85173,48
18287,85040A,48
18287,85039B,72
18287,85039B,48
18287,85039A,48


In [121]:
%%sparksql
SELECT CustomerId, stockCode, sum(Quantity) FROM dfNoNull
GROUP BY customerId, stockCode GROUPING SETS((customerId, stockCode),())
ORDER BY CustomerId DESC, stockCode DESC limit 10

customerId,stockCode,sum(Quantity)
18287,85173,48
18287,85040A,48
18287,85039B,120
18287,85039A,96
18287,84920,4
18287,84584,6
18287,84507C,6
18287,72351B,24
18287,72351A,24
18287,72349B,60


In [123]:
%%sparksql
DROP TABLE IF EXISTS dealer;

In [132]:
%%sparksql
select sum(quantity) from dfNoNull

sum(quantity)
5176450


In [131]:
%%sparksql
SELECT CustomerId, stockCode, sum(Quantity) FROM dfNoNull
GROUP BY customerId, stockCode GROUPING SETS((customerId, stockCode),())
ORDER BY CustomerId DESC NULLS FIRST, stockCode DESC NULLS FIRST

only showing top 20 row(s)


customerId,stockCode,sum(Quantity)
null,null,5176450
null,m,1
null,gift_0001_50,4
null,gift_0001_40,3
null,gift_0001_30,37
null,gift_0001_20,20
null,gift_0001_10,39
null,S,-59
null,POST,3351
null,M,-20


In [161]:
%%sparksql
SELECT CustomerId, stockCode, sum(Quantity) as sum_quantity FROM dfNoNull
GROUP BY customerId, stockCode with ROLLUP
order by sum(quantity) desc
--ORDER BY CustomerId, stockCode

only showing top 20 row(s)


customerId,stockCode,sum_quantity
null,null,5176450
null,null,269562
14646,null,196719
12415,null,77242
14911,null,77180
17450,null,69029
18102,null,64122
17511,null,63012
13694,null,61803
14298,null,58021


In [162]:
%%sparksql
select sum(q1.sum_quantity) from (SELECT CustomerId, stockCode, sum(Quantity) as sum_quantity FROM dfNoNull
GROUP BY customerId, stockCode with ROLLUP
order by sum(quantity) desc) q1 where q1.customerid is not null and q1.stockcode is null 

sum(sum_quantity)
4906888


In [151]:
rolledUpDF = dfNoNull.rollup("Date", "Country").agg(fn.sum("Quantity"))\
.selectExpr("Date", "Country", "`sum(Quantity)` as total_quantity")\
.orderBy("Date")
rolledUpDF.show()


+----------+--------------+--------------+
|      Date|       Country|total_quantity|
+----------+--------------+--------------+
|      NULL|          NULL|       5176450|
|2010-12-01|     Australia|           107|
|2010-12-01|United Kingdom|         23949|
|2010-12-01|        France|           449|
|2010-12-01|        Norway|          1852|
|2010-12-01|          NULL|         26814|
|2010-12-01|       Germany|           117|
|2010-12-01|          EIRE|           243|
|2010-12-01|   Netherlands|            97|
|2010-12-02|          NULL|         21023|
|2010-12-02|United Kingdom|         20873|
|2010-12-02|       Germany|           146|
|2010-12-02|          EIRE|             4|
|2010-12-03|          NULL|         14830|
|2010-12-03|      Portugal|            65|
|2010-12-03|         Spain|           400|
|2010-12-03|United Kingdom|         10439|
|2010-12-03|   Switzerland|           110|
|2010-12-03|       Germany|           170|
|2010-12-03|          EIRE|          2575|
+----------

In [152]:
rolledUpDF.where('Country is NULL').show()

+----------+-------+--------------+
|      Date|Country|total_quantity|
+----------+-------+--------------+
|      NULL|   NULL|       5176450|
|2010-12-01|   NULL|         26814|
|2010-12-02|   NULL|         21023|
|2010-12-03|   NULL|         14830|
|2010-12-05|   NULL|         16395|
|2010-12-06|   NULL|         21419|
|2010-12-07|   NULL|         24995|
|2010-12-08|   NULL|         22741|
|2010-12-09|   NULL|         18431|
|2010-12-10|   NULL|         20297|
|2010-12-12|   NULL|         10565|
|2010-12-13|   NULL|         17623|
|2010-12-14|   NULL|         20098|
|2010-12-15|   NULL|         18229|
|2010-12-16|   NULL|         29632|
|2010-12-17|   NULL|         16069|
|2010-12-19|   NULL|          3795|
|2010-12-20|   NULL|         14965|
|2010-12-21|   NULL|         15467|
|2010-12-22|   NULL|          3192|
+----------+-------+--------------+
only showing top 20 rows



In [153]:
rolledUpDF.where('Date is NULL').show()

+----+-------+--------------+
|Date|Country|total_quantity|
+----+-------+--------------+
|NULL|   NULL|       5176450|
+----+-------+--------------+



In [141]:
%%sparksql
select * from (SELECT CustomerId, stockCode, sum(Quantity) as sum_quantity FROM dfNoNull
GROUP BY customerId, stockCode with ROLLUP 
ORDER BY CustomerId DESC NULLS FIRST, stockCode DESC NULLS FIRST) q1 where q1.stockcode is NULL and customerid is NULL order by q1.sum_quantity desc

customerId,stockCode,sum_quantity
null,null,5176450
null,null,269562


In [165]:
cube_df = dfNoNull.cube("Date", "Country").agg(fn.sum(fn.col("Quantity")))\
.select("Date", "Country", "sum(Quantity)").orderBy("Date")
cube_df.show(truncate=False)

+----+---------------+-------------+
|Date|Country        |sum(Quantity)|
+----+---------------+-------------+
|NULL|Cyprus         |6317         |
|NULL|Belgium        |23152        |
|NULL|Austria        |4827         |
|NULL|Germany        |117448       |
|NULL|Lithuania      |652          |
|NULL|Poland         |3653         |
|NULL|Iceland        |2458         |
|NULL|Australia      |83653        |
|NULL|Finland        |10666        |
|NULL|Norway         |19247        |
|NULL|Italy          |7999         |
|NULL|EIRE           |142637       |
|NULL|United Kingdom |4263829      |
|NULL|Spain          |26824        |
|NULL|Lebanon        |386          |
|NULL|Bahrain        |260          |
|NULL|NULL           |5176450      |
|NULL|Israel         |4353         |
|NULL|Channel Islands|9479         |
|NULL|Switzerland    |30325        |
+----+---------------+-------------+
only showing top 20 rows



In [167]:
cube_df.where('date is null and country is null').show()

+----+-------+-------------+
|Date|Country|sum(Quantity)|
+----+-------+-------------+
|NULL|   NULL|      5176450|
+----+-------+-------------+



<h2>Grouping Metadata</h2>

<p>Sometimes when using cubes and rollups, you want to be able to query the aggregation levels so
that you can easily filter them down accordingly. We can do this by using the grouping_id,
which gives us a column specifying the level of aggregation that we have in our result set. The
query in the example that follows returns four distinct grouping IDs:</p>

<br>GroupingID | Description</br>
- 3          This will appear for the highest-level aggregation, which will gives us the total quantity
           regardless of customerId and stockCode.
- 2          This will appear for all aggregations of individual stock codes. This gives us the total quantity
           per stock code, regardless of customer.
- 1          This will give us the total quantity on a per-customer basis, regardless of item purchased.
- 0          This will give us the total quantity for individual customerId and stockCode combinations.

In [171]:
dfNoNull.cube("Date", "Country").agg(fn.grouping_id(), fn.sum(fn.col("Quantity")))\
.orderBy("grouping_id()").show()

+----------+--------------+-------------+-------------+
|      Date|       Country|grouping_id()|sum(Quantity)|
+----------+--------------+-------------+-------------+
|2010-12-17|      Portugal|            0|          193|
|2011-01-05|       Bahrain|            0|          -54|
|2010-12-07|United Kingdom|            0|        23769|
|2011-01-20|          EIRE|            0|         1332|
|2011-01-27|        France|            0|           -9|
|2011-01-19|         Spain|            0|           48|
|2011-01-10|         Spain|            0|          643|
|2010-12-16|       Finland|            0|         1254|
|2010-12-10|          EIRE|            0|          379|
|2011-01-06|         Spain|            0|           -1|
|2010-12-22|       Belgium|            0|          754|
|2010-12-03|      Portugal|            0|           65|
|2010-12-07|       Iceland|            0|          319|
|2010-12-14|     Australia|            0|          -13|
|2011-01-24|     Hong Kong|            0|       

In [172]:
dfNoNull.cube("Date", "Country").agg(fn.grouping_id(), fn.sum(fn.col("Quantity")))\
.orderBy(fn.desc("grouping_id()")).where('grouping_id() = 1').show()

+----------+-------+-------------+-------------+
|      Date|Country|grouping_id()|sum(Quantity)|
+----------+-------+-------------+-------------+
|2010-12-17|   NULL|            1|        16069|
|2010-12-14|   NULL|            1|        20098|
|2011-01-06|   NULL|            1|        22461|
|2011-01-11|   NULL|            1|        29093|
|2010-12-19|   NULL|            1|         3795|
|2011-01-24|   NULL|            1|        11910|
|2011-01-13|   NULL|            1|        10114|
|2010-12-03|   NULL|            1|        14830|
|2010-12-23|   NULL|            1|         5648|
|2011-01-12|   NULL|            1|        10133|
|2010-12-09|   NULL|            1|        18431|
|2010-12-15|   NULL|            1|        18229|
|2011-01-18|   NULL|            1|         8664|
|2010-12-06|   NULL|            1|        21419|
|2011-01-19|   NULL|            1|        17108|
|2010-12-13|   NULL|            1|        17623|
|2011-01-14|   NULL|            1|        23091|
|2010-12-16|   NULL|

In [170]:
dfNoNull.cube("Date", "Country").agg(fn.grouping_id(), fn.sum(fn.col("Quantity")))\
.orderBy(fn.desc("grouping_id()")).show()

+----+---------------+-------------+-------------+
|Date|        Country|grouping_id()|sum(Quantity)|
+----+---------------+-------------+-------------+
|NULL|           NULL|            3|      5176450|
|NULL|Channel Islands|            2|         9479|
|NULL|          Italy|            2|         7999|
|NULL|         France|            2|       110480|
|NULL|          Japan|            2|        25218|
|NULL|        Iceland|            2|         2458|
|NULL| United Kingdom|            2|      4263829|
|NULL|         Greece|            2|         1556|
|NULL|         Sweden|            2|        35637|
|NULL|      Australia|            2|        83653|
|NULL|         Norway|            2|        19247|
|NULL|      Lithuania|            2|          652|
|NULL|           EIRE|            2|       142637|
|NULL|      Singapore|            2|         5234|
|NULL|          Spain|            2|        26824|
|NULL|        Lebanon|            2|          386|
|NULL|        Bahrain|         

In [173]:
pivoted = dfWithDate.groupBy(fn.col('date')).pivot('country').sum()

In [180]:
pivoted.show(truncate=False)

+----------+-----------------------+------------------------+-------------------------+---------------------+----------------------+-----------------------+---------------------+----------------------+-----------------------+---------------------+----------------------+-----------------------+--------------------+---------------------+----------------------+--------------------+---------------------+----------------------+-----------------------------+------------------------------+-------------------------------+--------------------+---------------------+----------------------+----------------------------+-----------------------------+------------------------------+---------------------+----------------------+-----------------------+------------------+-------------------+--------------------+--------------------------------+---------------------------------+----------------------------------+---------------------+----------------------+-----------------------+--------------------+----

In [199]:
pivoted.where("date > '2011-12-05'").selectExpr('date', '`USA_sum(quantity)`').show()

+----------+-----------------+
|      date|USA_sum(quantity)|
+----------+-----------------+
|2011-12-06|             NULL|
|2011-12-09|             NULL|
|2011-12-08|             -196|
|2011-12-07|             NULL|
+----------+-----------------+



In [192]:
# spark = SparkSession.builder.config("spark.sql.catalogImplementation", "hive").enableHiveSupport().getOrCreate()

In [193]:
# %%sparksql
# create database if not exists ct

In [198]:
# %%sparksql
# CREATE TABLE ct.sampleTable (number Int, word String)


AnalysisException: [NOT_SUPPORTED_COMMAND_WITHOUT_HIVE_SUPPORT] CREATE Hive TABLE (AS SELECT) is not supported, if you want to enable it, please set "spark.sql.catalogImplementation" to "hive".;
'CreateTable `spark_catalog`.`ct`.`sampleTable`, org.apache.hadoop.hive.serde2.lazy.LazySimpleSerDe, ErrorIfExists


In [ ]:
for i in enumerate([1, 2, 3]):
    print(i)

(0, 1)
(1, 2)
(2, 3)


In [ ]:
stoi = {ch:i for i, ch in enumerate('abcdefghijklmnopqrstuvwxyz')}

In [ ]:
stoi

{'a': 0,
 'b': 1,
 'c': 2,
 'd': 3,
 'e': 4,
 'f': 5,
 'g': 6,
 'h': 7,
 'i': 8,
 'j': 9,
 'k': 10,
 'l': 11,
 'm': 12,
 'n': 13,
 'o': 14,
 'p': 15,
 'q': 16,
 'r': 17,
 's': 18,
 't': 19,
 'u': 20,
 'v': 21,
 'w': 22,
 'x': 23,
 'y': 24,
 'z': 25}

In [ ]:
itos = {i:ch for i, ch in enumerate('abcdefghijklmnopqrstuvwxyz')}

In [ ]:
itos

{0: 'a',
 1: 'b',
 2: 'c',
 3: 'd',
 4: 'e',
 5: 'f',
 6: 'g',
 7: 'h',
 8: 'i',
 9: 'j',
 10: 'k',
 11: 'l',
 12: 'm',
 13: 'n',
 14: 'o',
 15: 'p',
 16: 'q',
 17: 'r',
 18: 's',
 19: 't',
 20: 'u',
 21: 'v',
 22: 'w',
 23: 'x',
 24: 'y',
 25: 'z'}

In [ ]:
encode = [stoi[ch] for ch in 'hiithere']

In [ ]:
encode

[7, 8, 8, 19, 7, 4, 17, 4]

In [ ]:
decode = ''.join([itos[i] for i in encode])

In [ ]:
decode

'hiithere'